Imports

In [ ]:
import time
import os
from pathlib import Path
import ipywidgets as widgets
from enum import Enum
from pynq import Overlay, allocate
from IPython.display import display, clear_output

from axi.driver import SimpleAxiMasterDriver
from axi.enums import MemSize, MemOpStatus

Set up hardware and allocate buffer.

In [ ]:
print('Initializing hardware...')
BITSTREAM_PATH = '../overlay/demo.bit'

if not os.path.exists(BITSTREAM_PATH):
    raise FileNotFoundError(f'Bitstream not found at {BITSTREAM_PATH}. Run \'make bitstream\'.')
driver = SimpleAxiMasterDriver(Path(BITSTREAM_PATH))

print('Hardware initialized!')
print('Allocating buffer...')

# Allocate 1024 * 4B buffer
buffer = allocate(shape=(1024,), dtype='u4')
buffer_addr = buffer.physical_address

print(f'DDR buffer allocated at {hex(buffer_addr)}')

Set up UI.

In [ ]:
w_addr = widgets.Text(value=hex(buffer_addr), description="Addr:")
w_data = widgets.Text(value="0xDEADBEEF", description="Data:")
btn_run = widgets.Button(description="Write Data", button_style='warning')
out_log = widgets.Output()

def on_click(b):
    with out_log:
        addr = int(w_addr.value, 16)
        data = int(w_data.value, 16)
        
        result = driver._execute(cmd=1, size=2, addr=addr, data=data)
        
        if result.status != MemOpStatus.DONE:
            print("Error!")
        else:
            print("Success!")

btn_run.on_click(on_click)
display(widgets.VBox([w_addr, w_data, btn_run, out_log]))